In [1]:
import numpy as np
import pickle as cPickle
import os
import gzip

# First exercise: Classifying MNIST with MLPs
In this exercise you will implement a Neural Network (or MLP) and classify the MNIST digits with it.
MNIST is a "well hung" dataset that has been used a lot over the years to benchmark different classification algorithms. 
To learn more about it have a look here: http://yann.lecun.com/exdb/mnist/ .

# Data Loading
We first define a function for downloading and loading MNIST.
**WARNING**: Executing it will obviously use up some space on your machine ;). 

In [2]:
def mnist(datasets_dir='./data'):
    if not os.path.exists(datasets_dir):
        os.mkdir(datasets_dir)
    data_file = os.path.join(datasets_dir, 'mnist.pkl.gz')
    if not os.path.exists(data_file):
        print('... downloading MNIST from the web')
        try:
            import urllib
            urllib.urlretrieve('http://google.com')
        except AttributeError:
            import urllib.request as urllib
        url = 'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz'
        urllib.urlretrieve(url, data_file)

    print('... loading data')
    # Load the dataset
    f = gzip.open(data_file, 'rb')
    try:
        train_set, valid_set, test_set = cPickle.load(f, encoding="latin1")
    except TypeError:
        train_set, valid_set, test_set = cPickle.load(f)
    f.close()

    test_x, test_y = test_set
    test_x = test_x.astype('float32')
    test_x = test_x.astype('float32').reshape(test_x.shape[0], 1, 28, 28)
    test_y = test_y.astype('int32')
    valid_x, valid_y = valid_set
    valid_x = valid_x.astype('float32')
    valid_x = valid_x.astype('float32').reshape(valid_x.shape[0], 1, 28, 28)
    valid_y = valid_y.astype('int32')
    train_x, train_y = train_set
    train_x = train_x.astype('float32').reshape(train_x.shape[0], 1, 28, 28)
    train_y = train_y.astype('int32')
    rval = [(train_x, train_y), (valid_x, valid_y), (test_x, test_y)]
    print('... done loading data')
    return rval

# Neural Network Layers
We now define "bare bone" neural network layers.
The parts marked with **TODO** are where you should finish the implementation!
Conceptually we will implement the layers as follows:

Each layer has a constructor that takes an input layer plus some additional arguments such as layer size and the activation function name. The layer then uses the provided input layer to compute the layer dimensions, weight shapes, etc. and setup all auxilliary variables.

Each layer then has to provide three functions (as defined in the Layer class below): *output_shape()*, *fprop()* and *brop()*. The output_shape function is used to figure out the shape for the next layer and the *fprop()/bprop()* functions are used to compute forward and backward passes through the network.

In [3]:
# start by defining simple helpers
def sigmoid(x):
    return 1.0/(1.0+np.exp(-x))

def sigmoid_d(x):
    return sigmoid(x)*(1.-sigmoid(x))

def tanh(x):
    return np.tanh(x)

def tanh_d(x):
    return 1. - tanh(x)**2

def relu(x):
    return np.maximum(0.0, x)

def relu_d(x):
    dx = np.zeros_like(x)
    dx[x > 0] = 1.
    return dx

def softmax(x, axis=1):
    # to make the softmax a "safe" operation we will 
    # first subtract the maximum along the specified axis
    # so that np.exp(x) does not blow up!
    # Note that this does not change the output.
    x_max = np.max(x, axis=axis, keepdims=True)
    x_safe = x - x_max
    e_x = np.exp(x_safe)
    return e_x / np.sum(e_x, axis=axis, keepdims=True)

def one_hot(labels):
    """this creates a one hot encoding from a flat vector:
    i.e. given y = [0,2,1]
     it creates y_one_hot = [[1,0,0], [0,0,1], [0,1,0]]
    """
    classes = np.unique(labels)
    n_classes = classes.size
    one_hot_labels = np.zeros(labels.shape + (n_classes,))
    for c in classes:
        one_hot_labels[labels == c, c] = 1
    return one_hot_labels

def unhot(one_hot_labels):
    """ Invert a one hot encoding, creating a flat vector """
    return np.argmax(one_hot_labels, axis=-1)

# then define an activation function class
class Activation(object):
    
    def __init__(self, tname):
        if tname == 'sigmoid':
            self.act = sigmoid
            self.act_d = sigmoid_d
        elif tname == 'tanh':
            self.act = tanh
            self.act_d = tanh_d
        elif tname == 'relu':
            self.act = relu
            self.act_d = relu_d
        else:
            raise ValueError('Invalid activation function.')
            
    def fprop(self, input):
        # we need to remember the last input
        # so that we can calculate the derivative with respect
        # to it later on
        self.last_input = input
        return self.act(input)
    
    def bprop(self, output_grad):
        return np.multiply(output_grad,self.act_d(self.last_input))

# define a base class for layers
class Layer(object):
    
    def fprop(self, input):
        """ Calculate layer output for given input 
            (forward propagation). 
        """
        raise NotImplementedError('This is an interface class, please use a derived instance')

    def bprop(self, output_grad):
        """ Calculate input gradient and gradient 
            with respect to weights and bias (backpropagation). 
        """
        raise NotImplementedError('This is an interface class, please use a derived instance')

    def output_size(self):
        """ Calculate size of this layer's output.
        input_shape[0] is the number of samples in the input.
        input_shape[1:] is the shape of the feature.
        """
        raise NotImplementedError('This is an interface class, please use a derived instance')

# define a base class for loss outputs
# an output layer can then simply be derived
# from both Layer and Loss 
class Loss(object):

    def loss(self, output, output_net):
        """ Calculate mean loss given real output and network output. """
        raise NotImplementedError('This is an interface class, please use a derived instance')

    def input_grad(self, output, output_net):
        """ Calculate input gradient real output and network output. """
        raise NotImplementedError('This is an interface class, please use a derived instance')

# define a base class for parameterized things        
class Parameterized(object):
    
    def params(self):
        """ Return parameters (by reference) """
        raise NotImplementedError('This is an interface class, please use a derived instance')
    
    def grad_params(self):
        """ Return accumulated gradient with respect to params. """
        raise NotImplementedError('This is an interface class, please use a derived instance')

# define a container for providing input to the network
class InputLayer(Layer):
    
    def __init__(self, input_shape):
        if not isinstance(input_shape, tuple):
            raise ValueError("InputLayer requires input_shape as a tuple")
        self.input_shape = input_shape

    def output_size(self):
        return self.input_shape
    
    def fprop(self, input):
        return input
    
    def bprop(self, output_grad):
        return output_grad
        
class FullyConnectedLayer(Layer, Parameterized):
    """ A standard fully connected hidden layer, as discussed in the lecture.
    """
    
    def __init__(self, input_layer, num_units, 
                 init_stddev, activation_fun=Activation('relu')):
        self.num_units = num_units
        self.activation_fun = activation_fun
        # the input shape will be of size (batch_size, num_units_prev) 
        # where num_units_prev is the number of units in the input 
        # (previous) layer
        self.input_shape = input_layer.output_size()
        # TODO ################################
        # TODO: implement weight initialization !DONE
        # TODO ################################
        # this is the weight matrix it should have shape: (num_units_prev, num_units)
        
        # We initialize the weights with a Gaussian distribution
        self.W = np.random.standard_normal((self.input_shape[1], self.num_units)) * init_stddev
        print("initialized W shape")
        print(self.W.shape)
        # and this is the bias vector of shape: (num_units)
        # We intitialize the bias terms with a very small number e.g. 0.1
        self.b = np.ones(self.num_units) * 0.1
        # create dummy variables for parameter gradients
        # no need to change these here!
        self.dW = None
        self.db = None
    
    def output_size(self):
        return (self.input_shape[0], self.num_units)
    
    def fprop(self, input):
        # TODO ################################################
        # TODO: implement forward propagation
        # NOTE: you should also handle the case were 
        #       activation_fun is None (meaning no activation)
        #       then this is simply a linear layer
        # TODO ################################################
        # you again want to cache the last_input for the bprop
        # implementation below!
        self.last_input = input
        # NOTE: Use numpy dot product
        # h = t(a), a = W * x + b
        # compute activation
#         print("input shape")
#         print(input.shape)
#         print("W shape")
#         print(self.W.shape)
#         print("b shape")
#         print(self.b.shape)
        A = input.dot(self.W) + self.b
        # check for activation function
        # if none, return the activations as they are
        if self.activation_fun == None:
            return A
        # else compute the activation function using the activations
        H = self.activation_fun.fprop(A)
        
        # return the computed activation function
        return H
        
    def bprop(self, output_grad):
        """ Calculate input gradient (backpropagation). """
        # TODO ################################
        # TODO: implement backward propagation
        # TODO ###############################
        
        # HINT: you may have to divide dW and db by n
        #       to make gradient checking work 
        #       OR you divide the gradient in the output layer by n
        n = output_grad.shape[0]
        # accumulate gradient wrt. the parameters first
        # we will need to store these to later update
        # the network after a few forward backward passes
        # NOTE: you should also handle the case were 
        #       activation_fun is None (meaning no activation)
        # the gradient wrt. W should be stored as self.dW
        # the gradient wrt. b should be stored as selfdb
        # NOTE: self.dW is also a numpy dot product
#         print("output shape:")
#         print(self.output_size())
#         print("output grad:")
#         print(output_grad.shape)
        if self.activation_fun == None:
            hidden_act_grad = output_grad
        else:
            hidden_act_grad = self.activation_fun.bprop(output_grad)
#         self.dW = np.true_divide(self.last_input.T.dot(hidden_act_grad), n) #FIXED!
#         self.db = np.true_divide(hidden_act_grad.sum(axis=0, keepdims=True), n) #FIXED!
        self.dW = self.last_input.T.dot(hidden_act_grad) / n #FIXED!
        self.db = hidden_act_grad.sum(axis=0, keepdims=True) / n #FIXED!
#         print("hidden act shape:")
#         print(hidden_act_grad.shape)
#         print("W shape:")
#         print(self.W.shape)
#         print("b shape:")
#         print(self.b.shape)
#         print("last input shape:")
#         print(self.last_input.shape)
        # the gradient wrt. the input should be calculated here
        grad_input = hidden_act_grad.dot(self.W.T) #FIXED
        
        return grad_input 
        
    def params(self):
        return self.W, self.b

    def grad_params(self):
        return self.dW, self.db

# finally we specify the interface for output layers 
# which are layers that also have a loss function
# we will implement two output layers:
#  a Linear, and Softmax (Logistic Regression) layer
# The difference between output layers and and normal 
# layers is that they will be called to compute the gradient
# of the loss through input_grad(). bprop will never 
# be called on them!
class LinearOutput(Layer, Loss):
    """ A simple linear output layer that  
        uses a squared loss (e.g. should be used for regression)
    """
    def __init__(self, input_layer):
        self.input_shape = input_layer.output_size()
        
    def output_size(self):
        return (1,)
        
    def fprop(self, input):
        return input

    def bprop(self, output_grad):
        raise NotImplementedError(
            'LinearOutput should only be used as the last layer of a Network'
            + ' bprop() should thus never be called on it!'
        )
    
    def input_grad(self, Y, Y_pred):
        # TODO ####################################
        # TODO: implement gradient of squared loss  DONE!
        # TODO ####################################
        n_samples = float(Y.shape[0])
        return (Y_pred - Y) / n_samples

    def loss(self, Y, Y_pred):
        loss = 0.5 * np.square(Y_pred - Y)
        return np.mean(np.sum(loss, axis=1))

class SoftmaxOutput(Layer, Loss):
    """ A softmax output layer that calculates 
        the negative log likelihood as loss
        and should be used for classification.
    """
    
    def __init__(self, input_layer):
        self.input_shape = input_layer.output_size()
        
    def output_size(self):
        return (1,)
    
    def fprop(self, input):
        return softmax(input)
    
    def bprop(self, output_grad):
        raise NotImplementedError(
            'SoftmaxOutput should only be used as the last layer of a Network'
            + ' bprop() should thus never be called on it!'
        )
    
    def input_grad(self, Y, Y_pred):
        # TODO #######################################################
        # TODO: implement gradient of the negative log likelihood loss
        # TODO #######################################################
        # HINT: since this would involve taking the log 
        #       of the softmax (which is np.exp(x)/np.sum(x, axis=1))
        #       this gradient computation can be simplified a lot,
        #       you may find a connection to the LinearOutput layer!
        #n_samples = float(Y.shape[0])
        return -(Y - Y_pred) 

    def loss(self, Y, Y_pred):
        # Assume one-hot encoding of Y
        out = Y_pred
        # to make the loss numerically stable 
        # you should add an epsilon in the log ;)
        eps = 1e-10
        # TODO ####################################
        # calculate negative log likelihood
        # TODO ####################################
        return -np.sum(np.log(out[Y == 1]) + eps) / len(out)
        

# Neural Network class
With all layers in place (and properly implemented by you) we can finally define a neural network.
For our purposes a neural network is simply a collection of layers which we will cycle through and on which we will call fprop and bprop to compute partial derivatives with respect to the input and the parameters.

Pay special attention to the *check_gradients()* function in which you should implement automatic differentiation. This function will become your best friend when checking the correctness of your implementation.

In [4]:

class NeuralNetwork:
    """ Our Neural Network container class.
    """
    def __init__(self, layers):
        self.layers = layers
        
    def _loss(self, X, Y):
        Y_pred = self.predict(X)
        return self.layers[-1].loss(Y, Y_pred)

    def predict(self, X):
        """ Calculate an output Y for the given input X. """
        # TODO ##########################################
        # TODO: implement forward pass through all layers  DONE!
        # TODO ##########################################
        num_layers = len(self.layers)
        Y_pred = self.layers[0].fprop(X)
        for i in range(1,num_layers):
            Y_pred = self.layers[i].fprop(Y_pred)
        return Y_pred
    
    def backpropagate(self, Y, Y_pred, upto=0):
        """ Backpropagation of partial derivatives through 
            the complete network up to layer 'upto'
        """
        next_grad = self.layers[-1].input_grad(Y, Y_pred)
        num_layers = len(self.layers)
#         print("upto:")
#         print(upto)
#         print("num_layers")
#         print(num_layers)
        # TODO ##########################################
        # TODO: implement backward pass through all layers  DONE!
        # TODO ##########################################
        for i in range(num_layers-2,upto-1,-1):
#             print("inside for")
#             print(i)
            next_grad = self.layers[i].bprop(next_grad)
#             print("layer W: ")
#             print(self.layers[-1-i].W.shape)
#             print("layer b: ")
#             print(self.layers[-1-i].b.shape)
        return next_grad
    
    def classification_error(self, X, Y):
        """ Calculate error on the given data 
            assuming they are classes that should be predicted. 
        """
        Y_pred = unhot(self.predict(X))
        error = Y_pred != Y
        return np.mean(error)
    
    def sgd_epoch(self, X, Y, learning_rate, batch_size):
        n_samples = X.shape[0]
        n_batches = n_samples // batch_size
        for b in range(n_batches):
            # TODO #####################################
            # Implement stochastic gradient descent here  DONE!
            # TODO #####################################
            # start by extracting a batch from X and Y
            # (you can assume the inputs are already shuffled)

            # TODO: then forward and backward propagation + updates
            # HINT: layer.params() returns parameters *by reference*
            #       so you can easily update in-place
            
            # Determine the sample slice for this minibatch
            batch_start = b * batch_size
            batch_end = (b + 1) * batch_size
        
            # Extract the batch from the training set
            X_batch = X[batch_start:batch_end]
            Y_batch = Y[batch_start:batch_end]
            
            # Compute forward and backward pass to calculate the gradients
            Y_batch_pred = self.predict(X_batch)
            last_gradient = self.backpropagate(Y_batch, Y_batch_pred)
            
            # Update the weights
            num_layers = len(self.layers)
            for i in range(1,num_layers-1):
                #W, b = self.layers(i).params()
                dW, db = self.layers[i].grad_params()
                self.layers[i].W = self.layers[i].W - learning_rate * dW
                self.layers[i].b = self.layers[i].b - learning_rate * db
            # unnecessary for this implementation:
            # compute loss
            # Remember the loss of this batch for later
            #loss += self._loss(X_batch, Y_batch)
        #return loss / n_batches
    
    def adam_epoch(self, X, Y, learning_rate, batch_size, t):
        # Implement adam
        t += 1
        # Hyperparameters Adam
        grad_decay = 0.9
        grad_squared_decay = 0.95
        delta = 1e-8
        
        # create a list with all the parameters from the network so that we
        # can extract their sizes for our running means bookkeeping
        num_layers = len(self.layers)
        parameters = []
        for p in range(1,num_layers-1):
            W, b = self.layers[p].params()
            parameters.append(W)
            parameters.append(b)
        # Create the running means for the gradients and squared gradients
        # with list comprehensions. This is a quick notation to loop through
        # every element in the parameters list
        s = [np.ones_like(p)*0.001 for p in parameters]
#         print("s initialized: ")
#         print(s)
        r = [np.ones_like(p)*0.001 for p in parameters]
        # determine number of batches
        n_samples = X.shape[0]
        n_batches = n_samples // batch_size
        for b in range(n_batches):
#             print("batch")
#             print(b)
            # Determine the sample slice for this minibatch
            batch_start = b * batch_size
            batch_end = (b + 1) * batch_size

            # Extract the batch from the training set
            X_batch = X[batch_start:batch_end]
            Y_batch = Y[batch_start:batch_end]

            # Compute forward and backward pass to calculate the gradients
            Y_batch_pred = self.predict(X_batch)
            last_gradient = self.backpropagate(Y_batch, Y_batch_pred)

            # Update the weights
            num_layers = len(self.layers)
            # define indices for the two bookkeeping vectors for easier access
            indx_W = 0
            indx_b = 1
            for i in range(1,num_layers-1):
                dW, db = self.layers[i].grad_params()
                # update the running means
                s[indx_W] = grad_decay * s[indx_W] + (1 - grad_decay) * dW
                s[indx_b] = grad_decay * s[indx_b] + (1 - grad_decay) * db
                r[indx_W] = grad_squared_decay * r[indx_W] + (1 - grad_squared_decay) * dW**2
                r[indx_b] = grad_squared_decay * r[indx_b] + (1 - grad_squared_decay) * db**2
                # Correct the initial bias
                s[indx_W] = s[indx_W] / (1 - grad_decay**t)
                s[indx_b] = s[indx_b] / (1 - grad_decay**t)
                r[indx_W] = r[indx_W] / (1 - grad_squared_decay**t)
                r[indx_b] = r[indx_b] / (1 - grad_squared_decay**t)
                # update weights with ratio of the means
                self.layers[i].W = self.layers[i].W - learning_rate * s[indx_W] / (np.sqrt(r[indx_W] + delta))
#                 print("old b shape")
#                 print(self.layers[i].b.shape)
                self.layers[i].b = self.layers[i].b - learning_rate * s[indx_b] / (np.sqrt(r[indx_b] + delta))
#                 print("iteration: ")
#                 print(i)
#                 print("W shape")
#                 print(self.layers[i].W.shape)
#                 print("b shape")
#                 print(self.layers[i].b.shape)
                indx_W += 2
                indx_b += 2
        return t
    
    
    def gd_epoch(self, X, Y, learning_rate):
        # TODO ##################################################  DONE!
        # Implement batch gradient descent here
        # A few hints:
        #   There are two strategies you can follow:
        #   Either shove the whole dataset throught the network
        #   at once (which can be problematic for large datasets)
        #   or run through it batch wise as in the sgd approach
        #   and accumulate the gradients for all parameters as
        #   you go through the data. Either way you should then
        #   do one gradient step after you went through the
        #   complete dataset!
        # TODO ##################################################
        # I'll implement the first variant and hopefully it'll work
        # first compute the forward and backward pass for all input data 
        Y_pred = self.predict(X)
        last_gradient = self.backpropagate(Y, Y_pred)
        # update the weights
        num_layers = len(self.layers)
        for i in range(1,num_layers-1):
            #W, b = self.layers(i).params()
            dW, db = self.layers[i].grad_params()
            self.layers[i].W = self.layers[i].W - learning_rate * dW
            self.layers[i].b = self.layers[i].b - learning_rate * db
        # redundant for this implementation
        # return the los for this epoch
        # return self._loss(X, Y)
    
    
    
    def train(self, X, Y, X_valid, Y_valid, learning_rate=0.1, max_epochs=100, batch_size=64,
              descent_type="sgd", y_one_hot=True):
        # Bookkeeping for Adam
        t = 0
        # define vectors for bookkeeping
        training_loss = np.ones(max_epochs+1) * np.nan
        validation_loss_vec = np.ones(max_epochs+1) * np.nan
        """ Train network on the given data. """
        n_samples = X.shape[0]
        n_batches = n_samples // batch_size
        if y_one_hot:
            Y_train = one_hot(Y)
            Y_valid_one = one_hot(Y_valid)
        else:
            Y_train = Y
        print("... starting training")
        
        for e in range(max_epochs+1):
            if descent_type == "sgd":
                self.sgd_epoch(X, Y_train, learning_rate, batch_size)
            elif descent_type == "gd":
                self.gd_epoch(X, Y_train, learning_rate)
            elif descent_type == "adam":
                t = self.adam_epoch(X, Y_train, learning_rate, batch_size, t)
            else:
                raise NotImplementedError("Unknown gradient descent type {}".format(descent_type))

            # Output error on the training data
            train_loss = self._loss(X, Y_train)
            train_error = self.classification_error(X, Y)
            training_loss[e] = train_loss
            validation_error = self.classification_error(X_valid, Y_valid)
            validation_loss = self._loss(X_valid, Y_valid_one)
#             print(validation_loss)
            validation_loss_vec[e] = validation_loss
            print('epoch {:.4f}, loss {:.4f}, train error {:.4f}, validation error {:.4f}'.format(e, train_loss, train_error, validation_error))
            # TODO ##################################################
            # compute error on validation data:
            # simply make the function take validation data as input  DONE! (above)
            # and then compute errors here and print them
            # TODO ##################################################
        return training_loss, validation_loss_vec
            
    
    def check_gradients(self, X, Y):
        """ Helper function to test the parameter gradients for
        correctness. """
        for l, layer in enumerate(self.layers):
            if isinstance(layer, Parameterized):
                print('checking gradient for layer {}'.format(l))
                for p, param in enumerate(layer.params()):
                    # we iterate through all parameters
                    param_shape = param.shape
                    ########################Prints for debugging
#                     print("W: ")
#                     print(layer.W.shape)
#                     print("b: ")
#                     print(layer.b.shape)
                    
                    # define functions for conveniently swapping
                    # out parameters of this specific layer and 
                    # computing loss and gradient with these 
                    # changed parametrs
                    def output_given_params(param_new):
                        """ A function that will compute the output 
                            of the network given a set of parameters
                        """
                        # copy provided parameters
                        param[:] = np.reshape(param_new, param_shape)
                        # return computed loss
                        return self._loss(X, Y)

                    def grad_given_params(param_new):
                        """A function that will compute the gradient 
                           of the network given a set of parameters
                        """
                        # copy provided parameters
                        param[:] = np.reshape(param_new, param_shape)
                        # Forward propagation through the net
                        Y_pred = self.predict(X)
                        # Backpropagation of partial derivatives
                        self.backpropagate(Y, Y_pred, upto=l)
                        # return the computed gradient 
                        return np.ravel(self.layers[l].grad_params()[p])

                    # let the initial parameters be the ones that
                    # are currently placed in the network and flatten them
                    # to a vector for convenient comparisons, printing etc.
                    param_init = np.ravel(np.copy(param))
                    
                    # To debug you network's gradients use scipys
                    # gradient checking!
                    epsilon = 1e-5
                    import scipy.optimize
                    err = scipy.optimize.check_grad(output_given_params, 
                          grad_given_params, param_init) 
                    print('diff scipy {:.2e}'.format(err))
                    assert(err < epsilon)
                    
                    # reset the parameters to their initial values
                    param[:] = np.reshape(param_init, param_shape)

# Gradient Checking
After implementing everything it is always a good idea to setup some layers and perform gradient
checking on random data. **Note** that this is only an example! It is not a useful network architecture ;). We also expect you to play around with this to test all your implemented components.

In [5]:
input_shape = (5, 10)
n_labels = 6
layers = [InputLayer(input_shape)]

layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=15,
                init_stddev=0.1,
                activation_fun=Activation('relu')
))
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=6,
                init_stddev=0.1,
                activation_fun=Activation('tanh')
))
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=n_labels,
                init_stddev=0.1,
                activation_fun=Activation('relu')
))
layers.append(SoftmaxOutput(layers[-1]))
nn = NeuralNetwork(layers)
# for i in range(5):
#     print("layer: ")
#     print(i)
#     print(layers[i].input_shape)

initialized W shape
(10, 15)
initialized W shape
(15, 6)
initialized W shape
(6, 6)


In [6]:
# create random data
X = np.random.normal(size=input_shape)
# Y = np.random.normal(size=(1,1))
# and random labels
Y = np.zeros((input_shape[0], n_labels))
for i in range(Y.shape[0]):
    idx = np.random.randint(n_labels)
    Y[i, idx] = 1.
print(Y)

[[ 0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  0.  0.]]


In [7]:
nn.check_gradients(X, Y)

checking gradient for layer 1
diff scipy 1.51e-07
diff scipy 4.62e-08
checking gradient for layer 2
diff scipy 1.01e-07
diff scipy 2.97e-08
checking gradient for layer 3
diff scipy 8.44e-08
diff scipy 3.57e-08


# Training on MNIST
Finally we can let our network run on the MNIST dataset!

First load the data and reshape it.

In [8]:
# load
Dtrain, Dval, Dtest = mnist()
X_train, y_train = Dtrain
X_valid, y_valid = Dval

... loading data
... done loading data


*Dtrain* contains 50k images which are of size 28 x 28 pixels. Hence:

In [9]:
print("X_train shape: {}".format(np.shape(X_train)))
print("y_train shape: {}".format(np.shape(y_train)))

X_train shape: (50000, 1, 28, 28)
y_train shape: (50000,)


y_train will automatically be converted in the *train()* function to one_hot encoding.


But we need to reshape X_train, as our Network expects flat vectors of size 28*28 as input!

In [10]:
X_train = X_train.reshape(X_train.shape[0], -1)
print("Reshaped X_train size: {}".format(X_train.shape))
X_valid = X_valid.reshape((X_valid.shape[0], -1))
print("Reshaped X_valid size: {}".format(X_valid.shape))

Reshaped X_train size: (50000, 784)
Reshaped X_valid size: (10000, 784)


Ah, much better ;-)! 

Now we can finally really start training a Network!


I pre-defined a small Network for you below. Again This is not really a good default and will not produce state of the art results. Please play around with this a bit. See how different activation functions and training procedures (gd / sgd) affect the result.

In [15]:
import time

# Setup a small MLP / Neural Network
# we can set the first shape to None here to indicate that
# we will input a variable number inputs to the network
input_shape = (None, 28*28)
layers = [InputLayer(input_shape)]
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=300,
                init_stddev=0.01,
                activation_fun=Activation('relu')
))
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=400,
                init_stddev=0.01,
                activation_fun=Activation('relu')
))
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=10,
                init_stddev=0.01,
                # last layer has no nonlinearity 
                # (softmax will be applied in the output layer)
                activation_fun=None 
))
layers.append(SoftmaxOutput(layers[-1]))

nn = NeuralNetwork(layers)
# Train neural network
t0 = time.time()
nn.train(X_train, y_train, X_valid, y_valid, learning_rate=0.5, 
         max_epochs=20, batch_size=128, descent_type="sgd", y_one_hot=True)
t1 = time.time()
print('Duration: {:.1f}s'.format(t1-t0))

initialized W shape
(784, 300)
initialized W shape
(300, 400)
initialized W shape
(400, 10)
... starting training
epoch 0.0000, loss 0.2083, train error 0.0662, validation error 0.0605
epoch 1.0000, loss 0.1354, train error 0.0441, validation error 0.0447
epoch 2.0000, loss 0.0855, train error 0.0273, validation error 0.0334
epoch 3.0000, loss 0.0580, train error 0.0181, validation error 0.0268
epoch 4.0000, loss 0.0453, train error 0.0142, validation error 0.0251
epoch 5.0000, loss 0.0327, train error 0.0100, validation error 0.0229
epoch 6.0000, loss 0.0246, train error 0.0076, validation error 0.0214
epoch 7.0000, loss 0.0217, train error 0.0070, validation error 0.0215
epoch 8.0000, loss 0.0172, train error 0.0056, validation error 0.0209
epoch 9.0000, loss 0.0233, train error 0.0079, validation error 0.0240
epoch 10.0000, loss 0.0142, train error 0.0041, validation error 0.0208
epoch 11.0000, loss 0.0093, train error 0.0027, validation error 0.0201
epoch 12.0000, loss 0.0059, trai

# Figure out a reasonable Network that achieves good performance
As the last part of this task, setup a network that works well and gets reasonable accuracy, say ~ 1-3 percent error on the **validation set**. 
Train this network on the complete data and compute the **test error**. 

Visualize the validation loss and training loss for each iteration in a plot, e.g. using matplotlib

In [18]:
# Handle the test data
X_test, y_test = Dtest
# reshape X
X_test = X_test.reshape(X_test.shape[0], -1)
# Construct NN as in example before used for testing
input_shape = (None, 28*28)
layers = [InputLayer(input_shape)]
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=300,
                init_stddev=0.01,
                activation_fun=Activation('relu')
))
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=400,
                init_stddev=0.01,
                activation_fun=Activation('relu')
))
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=10,
                init_stddev=0.01,
                # last layer has no nonlinearity 
                # (softmax will be applied in the output layer)
                activation_fun=None 
))
layers.append(SoftmaxOutput(layers[-1]))

my_nn = NeuralNetwork(layers)
# Train neural network
t0 = time.time()
training_loss, validation_loss = my_nn.train(X_train, y_train, X_valid, y_valid,
                            learning_rate=0.5, max_epochs=17, batch_size=128, descent_type="sgd", y_one_hot=True)
t1 = time.time()
print('Duration: {:.1f}s'.format(t1-t0))

initialized W shape
(784, 300)
initialized W shape
(300, 400)
initialized W shape
(400, 10)
... starting training
epoch 0.0000, loss 0.2283, train error 0.0715, validation error 0.0653
epoch 1.0000, loss 0.1376, train error 0.0450, validation error 0.0451
epoch 2.0000, loss 0.0928, train error 0.0306, validation error 0.0352
epoch 3.0000, loss 0.0618, train error 0.0204, validation error 0.0272
epoch 4.0000, loss 0.0463, train error 0.0155, validation error 0.0245
epoch 5.0000, loss 0.0312, train error 0.0098, validation error 0.0216
epoch 6.0000, loss 0.0253, train error 0.0078, validation error 0.0206
epoch 7.0000, loss 0.1320, train error 0.0420, validation error 0.0449
epoch 8.0000, loss 0.1074, train error 0.0367, validation error 0.0437
epoch 9.0000, loss 0.0463, train error 0.0157, validation error 0.0264
epoch 10.0000, loss 0.0282, train error 0.0088, validation error 0.0231
epoch 11.0000, loss 0.0212, train error 0.0067, validation error 0.0221
epoch 12.0000, loss 0.0195, trai

In [19]:
import matplotlib as mpl
if float(mpl.__version__[:3]) >= 1.5:
    %matplotlib notebook
else:
    %matplotlib inline
    
import matplotlib.pyplot as plt
n_epochs = 17
# Plot the results
fig, ax = plt.subplots(1, 1)
ax.set_title('Training progress')
ax.set_xlim([0, n_epochs])
ax.set_ylim([0, 0.5])
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
def plot(ax):
    if ax.lines:
        ax.lines[0].set_ydata(training_loss)
        ax.lines[1].set_ydata(validation_loss)
        ax.set_ylim([0,  0.4])
    else:
        ax.plot(training_loss, c='b', label='Training loss')
        ax.plot(validation_loss, c='m', label='Validation loss')
        ax.legend(loc='lower left')
    fig.canvas.draw()
plot(ax)

<IPython.core.display.Javascript object>

In [21]:
# Compute test error
test_error = my_nn.classification_error(X_test, y_test)
print('Test error {:.3f} %'.format(test_error*100))
# Compue test error on previous nn
test_error = nn.classification_error(X_test, y_test)
print('Test error previous nn {:.3f} %'.format(test_error*100))

Test error 2.000 %
Test error previous nn 1.680 %
